In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math
from datetime import datetime
import time
import IPython
import os
import threading
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('error', category=UserWarning)

In [2]:
REGIONLIST= ["US", "EU", "UK"]
MODELLIST = ["64", "256", "512"]
ESTIMATELIST = ["Q1","Q2", "Q3"]

In [3]:
def AllPossibleQueues_Func():
  result = []
  for r in REGIONLIST:
    if (r == "ALL"):
      continue
    for m in MODELLIST:
      for e in ESTIMATELIST:
        result.append((r,m,e))
    
  return result

ALLPOSSIBLEQUEUES = AllPossibleQueues_Func()
print(ALLPOSSIBLEQUEUES)

[('US', '64', 'Q1'), ('US', '64', 'Q2'), ('US', '64', 'Q3'), ('US', '256', 'Q1'), ('US', '256', 'Q2'), ('US', '256', 'Q3'), ('US', '512', 'Q1'), ('US', '512', 'Q2'), ('US', '512', 'Q3'), ('EU', '64', 'Q1'), ('EU', '64', 'Q2'), ('EU', '64', 'Q3'), ('EU', '256', 'Q1'), ('EU', '256', 'Q2'), ('EU', '256', 'Q3'), ('EU', '512', 'Q1'), ('EU', '512', 'Q2'), ('EU', '512', 'Q3'), ('UK', '64', 'Q1'), ('UK', '64', 'Q2'), ('UK', '64', 'Q3'), ('UK', '256', 'Q1'), ('UK', '256', 'Q2'), ('UK', '256', 'Q3'), ('UK', '512', 'Q1'), ('UK', '512', 'Q2'), ('UK', '512', 'Q3')]


In [4]:
data = pd.read_excel('stdss.xlsx')

In [5]:
data.head()

,Timestamp,Region,Model,rtReserveTime or preorder-email time,Initial Valve Estimate,Your most recent pre-order estimated time,When did you receive your ready to order email?,Unnamed: 7
0,2022-02-25 13:21:39.764,US,256,1626454969,December,Q1,1.645813e+09,NaN
1,2022-02-09 14:50:45.171,UK,512,1635594996,After Q2,After Q2,NaN,NaN
2,2022-02-09 14:54:01.852,US,512,1626454856,December,Q1,NaN,NaN
3,2022-02-09 14:58:39.188,US,256,1644440188,Q1,Q2,NaN,NaN
4,2022-02-09 15:04:48.617,EU,256,1626478685,Q1,Q2,NaN,NaN


In [6]:
wrongInput = ((data["Initial Valve Estimate"] == "After Q2") | (data["Initial Valve Estimate"] == "Q3") | (data["Initial Valve Estimate"] == "After Q3"))
print(f"{np.sum(wrongInput)} entries removed for having wrong inputs in Estimate Column")
data.drop(data[wrongInput].index, inplace=True)

1329 entries removed for having wrong inputs in Estimate Column


In [7]:
data.drop(columns=["Timestamp", "Initial Valve Estimate", "Unnamed: 7"], inplace=True)
data.rename(columns = {"rtReserveTime or preorder-email time ": "Timestamp", "Your most recent pre-order estimated time":"ValveEstimate", "When did you receive your ready to order email?":"TrueOrderTimestamp"}, inplace=True)


namask = data["Region"].isna() | data["Model"].isna() | data["ValveEstimate"].isna() | data["Timestamp"].isna() | data["ValveEstimate"].isna()
print(f"{np.sum(namask)} entries removed for having NA on vital fields\n")
data.drop(data[namask].index,inplace=True)

20 entries removed for having NA on vital fields



In [8]:
wrongInput2 = (data["TrueOrderTimestamp"] < 1626516671)
print(f"{np.sum(wrongInput2)} entries removed for having wrong inputs in Estimate Column")
data.drop(data[wrongInput2].index, inplace=True)

0 entries removed for having wrong inputs in Estimate Column


In [9]:
#not valid for after Q2
#data = data[data["ValveEstimate"] != "After Q2"]
notEstimable = np.vectorize(lambda x: x not in ESTIMATELIST)
notEstimableMask = notEstimable(data["ValveEstimate"])
print(f"{np.sum(notEstimableMask)} entries removed for being \"After Q2\", \"After Q3\", or similar\n")
data.drop(data[notEstimableMask].index, inplace=True)

428 entries removed for being "After Q2", "After Q3", or similar



In [10]:
#entries supposedly happening after feb 25th 10am PDT
wrongUpdateMask = ((data["ValveEstimate"] == "Q1") | (data["ValveEstimate"] == "Q2") | (data["ValveEstimate"] == "Q3")) & (data["Timestamp"] >= 1645801200)
print(f"{np.sum(wrongUpdateMask)} entries removed for being after Februrary 25th 10am PDT\n")
data.drop(data[wrongUpdateMask].index, inplace=True)

144 entries removed for being after Februrary 25th 10am PDT



In [11]:
 #entries supposedly happening before july 16th 10am PDT
cretinMask = data["Timestamp"] < 1626454800
print(f"{np.sum(cretinMask)} entries removed for being before July 16th 10am PDT\n")
data.drop(data[cretinMask].index, inplace=True)

0 entries removed for being before July 16th 10am PDT



In [12]:
data.head()

,Region,Model,Timestamp,ValveEstimate,TrueOrderTimestamp
0,US,256,1626454969,Q1,1.645813e+09
2,US,512,1626454856,Q1,NaN
3,US,256,1644440188,Q2,NaN
4,EU,256,1626478685,Q2,NaN
5,US,512,1627423926,Q3,NaN


In [13]:
  @np.vectorize
  def digits(x):
    if (np.isnan(x)):
      return -1
    
    return int(np.log10(x))+1
  
  @np.vectorize
  def convertTimestampToSeconds(x):
    if (np.isnan(x)):
      return np.nan
    
    l = digits(x)
    if (l > 10 and (l - 10) % 3 == 0):
      m = (l - 10) // 3
      for _ in range(m):
        x = x // 1000
    
    return x

In [14]:
data["Timestamp"] = convertTimestampToSeconds(data["Timestamp"])
data["TrueOrderTimestamp"] = convertTimestampToSeconds(data["TrueOrderTimestamp"])

In [15]:
data.to_csv("Data.csv")

In [16]:
data.head()

,Region,Model,Timestamp,ValveEstimate,TrueOrderTimestamp
0,US,256,1626454969,Q1,1.645813e+09
2,US,512,1626454856,Q1,NaN
3,US,256,1644440188,Q2,NaN
4,EU,256,1626478685,Q2,NaN
5,US,512,1627423926,Q3,NaN


In [17]:
removeEmpty = data["TrueOrderTimestamp"].isna()
print(f"{np.sum(removeEmpty)} remove empty entries")
data.drop(data[removeEmpty].index, inplace=True)

4932 remove empty entries


In [18]:
data.dtypes

Region                 object
Model                   int64
Timestamp               int64
ValveEstimate          object
TrueOrderTimestamp    float64
dtype: object

In [19]:
data.head()

,Region,Model,Timestamp,ValveEstimate,TrueOrderTimestamp
0,US,256,1626454969,Q1,1.645813e+09
6,EU,512,1626454854,Q1,1.645813e+09
11,US,512,1626455651,Q2,1.650302e+09
30,EU,256,1626468720,Q2,1.650300e+09
34,UK,256,1626455083,Q1,1.646675e+09


In [20]:
s = (data.dtypes == 'object')
object_cols = list(s[s].index)

In [21]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

ordinal_enc = OrdinalEncoder()
data["Region"] = ordinal_enc.fit_transform(data[["Region"]])
data["ValveEstimate"] = ordinal_enc.fit_transform(data[["ValveEstimate"]])
data["Model"] = ordinal_enc.fit_transform(data[["Model"]])

In [22]:
data.head()

,Region,Model,Timestamp,ValveEstimate,TrueOrderTimestamp
0,2.0,1.0,1626454969,0.0,1.645813e+09
6,0.0,2.0,1626454854,0.0,1.645813e+09
11,2.0,2.0,1626455651,1.0,1.650302e+09
30,0.0,1.0,1626468720,1.0,1.650300e+09
34,1.0,1.0,1626455083,0.0,1.646675e+09


In [23]:
len(data)

599

In [24]:
y = data.iloc[:, -1]

X = data.iloc[:, 0:4]

X.to_csv("X.csv")
y.to_csv("y.csv")

In [25]:
#Do a test train split
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [26]:
#use linear regression to train the model and score the accuracy of the model
model = LinearRegression()
model = LinearRegression().fit(X_train, y_train)

In [27]:
r_sq = model.score(X_valid, y_valid)
print(r_sq)

-0.008362589112313135


In [28]:
from sklearn.metrics import mean_absolute_error

preds = model.predict(X_valid)
print(mean_absolute_error(y_valid, preds))


1842321524.3277347


In [33]:
#Region	Model	Timestamp	ValveEstimate
d = {'Region': 2.0, 'Model': 0.0, "TimeStamp": 1626460536, "ValveEstimate": 1.0}
df = pd.DataFrame(data=d, index=[0])

res  = model.predict(df)
print("{0:.0f}".format(res[0]))


1685656865


In [34]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

my_model = XGBRegressor()
my_pipeline = Pipeline(steps=[('model', my_model)])

my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, enable_categorical=False,
                              gamma=0, gpu_id=-1, importance_type=None,
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=12, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

In [35]:
r_sq = my_pipeline.score(X_valid, y_valid.values.ravel())
print(r_sq)

-0.008393769998462863


In [37]:
#Region	Model	Timestamp	ValveEstimate
d = {'Region': 2.0, 'Model': 0.0, "TimeStamp": 1649208320, "ValveEstimate": 1.0}
df = pd.DataFrame(data=d, index=[0])

res  = my_pipeline.predict(df)
print("{0:.0f}".format(res[0]))


1649194880
